In [1]:
###

In [2]:
!pip install ktrain

     |████████████████████████████████| 25.3 MB 48.0 MB/s 
     |████████████████████████████████| 6.8 MB 33.7 MB/s 
     |████████████████████████████████| 981 kB 12.7 MB/s 
     |████████████████████████████████| 263 kB 19.3 MB/s 
     |████████████████████████████████| 2.8 MB 36.9 MB/s 
     |████████████████████████████████| 1.2 MB 25.6 MB/s 
     |████████████████████████████████| 468 kB 41.1 MB/s 
     |████████████████████████████████| 61 kB 496 kB/s 
     |████████████████████████████████| 596 kB 40.8 MB/s 
     |████████████████████████████████| 3.3 MB 42.9 MB/s 
     |████████████████████████████████| 895 kB 41.2 MB/s 
  Created wheel for ktrain: filename=ktrain-0.28.3-py3-none-any.whl size=25292659 sha256=01bfcd5417a04b51c551fb6bef12e5a75d36d8be76a9342eecab129227fce412
  Stored in directory: /root/.cache/pip/wheels/6a/7e/c3/f46cdfc2b81c54424923b1405d7e670c35cacc11ada9a47b1c
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9929 sha256=1b31900bfe0bddd

In [3]:
!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git

Cloning into 'IMDB-Movie-Reviews-Large-Dataset-50k'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [4]:
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import tensorflow as tf

In [5]:
import os

In [6]:
os.listdir()

['.config', 'IMDB-Movie-Reviews-Large-Dataset-50k', 'sample_data']

In [7]:
df_train = pd.read_excel("/content/IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx",dtype = str)

In [8]:
df_test = pd.read_excel("/content/IMDB-Movie-Reviews-Large-Dataset-50k/test.xlsx",dtype = str)

In [9]:
text.print_text_classifiers()

fasttext: a fastText-like model [http://arxiv.org/pdf/1607.01759.pdf]
logreg: logistic regression using a trainable Embedding layer
nbsvm: NBSVM model [http://www.aclweb.org/anthology/P12-2018]
bigru: Bidirectional GRU with pretrained fasttext word vectors [https://fasttext.cc/docs/en/crawl-vectors.html]
standard_gru: simple 2-layer GRU with randomly initialized embeddings
bert: Bidirectional Encoder Representations from Transformers (BERT) from keras_bert [https://arxiv.org/abs/1810.04805]
distilbert: distilled, smaller, and faster BERT from Hugging Face transformers [https://arxiv.org/abs/1910.01108]


In [14]:
(train,val,preproc) = text.texts_from_df(train_df = df_train ,
                                         text_column="Reviews",
                                         label_columns="Sentiment",
                                         val_df = df_test,
                                         maxlen=256,
                                         preprocess_mode= "distilbert")

['neg', 'pos']
   neg  pos
0  1.0  0.0
1  1.0  0.0
2  1.0  0.0
3  1.0  0.0
4  1.0  0.0
['neg', 'pos']
   neg  pos
0  0.0  1.0
1  0.0  1.0
2  1.0  0.0
3  0.0  1.0
4  1.0  0.0
preprocessing train...
language: en
train sequence lengths:
	mean : 234
	95percentile : 598
	99percentile : 913


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 234
	95percentile : 598
	99percentile : 913


In [15]:
model = text.text_classifier(name = "distilbert",train_data = train,preproc = preproc)

Is Multi-Label? False
maxlen is 256
done.


In [16]:
learner = ktrain.get_learner(model = model,train_data = train,val_data = val,batch_size = 16)

In [18]:
learner.fit_onecycle(lr = 2e-5 , epochs = 2)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/2
1563/1563 [==============================] - 1729s 1s/step - loss: 0.3324 - accuracy: 0.8532 - val_loss: 0.2018 - val_accuracy: 0.9224
Epoch 2/2
1563/1563 [==============================] - 1726s 1s/step - loss: 0.1870 - accuracy: 0.9276 - val_loss: 0.0978 - val_accuracy: 0.9702


In [19]:
#####

In [20]:
#######

In [21]:
predictor = ktrain.get_predictor(learner.model,preproc)

In [24]:
predictor.save("/content/drive/MyDrive/distilbert/")

In [27]:
data = ["this movie is very bad","I watched LORD film it was great"]

In [28]:
predictor.predict(data)

/usr/local/lib/python3.7/dist-packages/ktrain/text/preprocessor.py:215: UserWarning: List or array of two texts supplied, so task being treated as text classification. If this is a sentence pair classification task, please cast to tuple.
  warnings.warn('List or array of two texts supplied, so task being treated as text classification. ' +\


['neg', 'pos']

In [29]:
predictor.get_classes()

['neg', 'pos']

In [30]:
predictor.predict_proba(data)

/usr/local/lib/python3.7/dist-packages/ktrain/text/preprocessor.py:215: UserWarning: List or array of two texts supplied, so task being treated as text classification. If this is a sentence pair classification task, please cast to tuple.
  warnings.warn('List or array of two texts supplied, so task being treated as text classification. ' +\


array([[0.9894134 , 0.01058665],
       [0.0162251 , 0.9837749 ]], dtype=float32)

In [31]:
predictor.predict(data,return_proba = True)

/usr/local/lib/python3.7/dist-packages/ktrain/text/preprocessor.py:215: UserWarning: List or array of two texts supplied, so task being treated as text classification. If this is a sentence pair classification task, please cast to tuple.
  warnings.warn('List or array of two texts supplied, so task being treated as text classification. ' +\


array([[0.9894134 , 0.01058665],
       [0.0162251 , 0.9837749 ]], dtype=float32)